1. Extração
- Nesta etapa, vamos extrair o CSV para a camada mais baixa (bronze) onde os dados brutos serão armazenados. 

In [ ]:
# Importando as bibliotecas
import pandas as pd
import requests
import os
from datetime import date, timedelta, datetime
from glob import glob
from zipfile import BadZipFile
import unicodedata
import locale


In [ ]:
def normalize_text(s):

    if isinstance(s, str):
        return (unicodedata.normalize("NFKD", s)   # separa acentos
                .encode("ASCII", "ignore")        # remove acentos
                .decode("utf-8")                  # volta p/ string
                .strip()                          # remove espaços extras
                .upper())                         # tudo maiúsculo
    return s


In [ ]:
from datetime import date, timedelta

def generate_weekly_files():
    today = date.today()

    
    last_week_end = today - timedelta(days=today.weekday() + 1)
    
    
    start = date(2025, 1, 5)
    end = start + timedelta(days=6)

    urls = []

    while end <= last_week_end:
        url = (
            f"https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/"
            f"precos/arquivos-lpc/{end.year}/"
            f"resumo_semanal_lpc_{start:%Y-%m-%d}_{end:%Y-%m-%d}.xlsx"
        )
        urls.append((url, start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")))

        
        start += timedelta(days=7)
        end += timedelta(days=7)

    return urls


In [ ]:
def collect_raw_data():
    current_year = date.today().year
    base_path = f"../data/bronze/{current_year}"
    os.makedirs(base_path, exist_ok=True)

    
    for urls, week_start, week_end in generate_weekly_files():
        date_end = datetime.strptime(week_end, "%Y-%m-%d").date()
        month = date_end.month

        
        mkdir_month = f"{base_path}/{month:02d}"
        os.makedirs(mkdir_month, exist_ok=True)

        
        file_path = f"{mkdir_month}/{week_start}_{week_end}.xlsx"

        
        if os.path.exists(file_path):
            
            continue

        
        resp = requests.get(urls)
        if resp.status_code == 200:
            with open(file_path, "wb") as f:
                f.write(resp.content)
            print(f"Arquivo salvo: {file_path}")
        else:
            print(f"Erro ao baixar {urls} -> status {resp.status_code}")


In [ ]:
collect_raw_data()

2. Transformação 
- Transformando os dados em CSV (dados consumíveis e consistentes)

In [ ]:
SHEETS = ["CAPITAIS", "MUNICIPIOS", "ESTADOS", "REGIOES", "BRASIL"]

def prepare_silver_structure():
    current_year = date.today().year
    silver_path = f"../data/silver/{current_year}/raw"
    os.makedirs(silver_path, exist_ok=True)

    
    bronze_files = glob(f"../data/bronze/{current_year}/*/*.xlsx")
    months = {os.path.basename(os.path.dirname(f)) for f in bronze_files}

    for month in months:
        for sheet in SHEETS:
            path = os.path.join(silver_path, month, sheet)
            os.makedirs(path, exist_ok=True)

    print(f"Estrutura Silver criada para {len(months)} meses e {len(SHEETS)} sheets")


In [ ]:
prepare_silver_structure()

In [ ]:
def convert_bronze_to_silver():
    current_year = date.today().year
    bronze_files = glob(f"../data/bronze/{current_year}/*/*.xlsx")
    silver_path = f"../data/silver/{current_year}/raw"

    for file in bronze_files:
        try:
            month = os.path.basename(os.path.dirname(file))
            xls = pd.ExcelFile(file, engine="openpyxl")

            for sheet in xls.sheet_names:
                try:
                    df_temp = pd.read_excel(xls, sheet_name=sheet)
                    header_row = df_temp.index[df_temp.iloc[:, 0] == "DATA INICIAL"][0]

                    df = pd.read_excel(xls, sheet_name=sheet, header=header_row, skiprows=1)

                    base_name = os.path.basename(file).replace(".xlsx", f"_{sheet.upper()}.csv")
                    csv_file = os.path.join(silver_path, month, sheet.upper(), base_name)

                    if os.path.exists(csv_file):
                        continue

                    df.to_csv(csv_file, index=False, encoding="utf-8-sig")
                    print(f"Arquivo convertido: {csv_file}")

                except Exception as e:
                    print(f"Erro na sheet {sheet} do arquivo {file}: {e}")

        except (BadZipFile, ValueError) as e:
            print(f"Arquivo inválido (pulado): {file} ({e})")
        except Exception as e:
            print(f"Erro inesperado em {file}: {e}")

In [ ]:
convert_bronze_to_silver()

- Verificar o nome de todas as colunas

In [ ]:
def list_unique_silver_columns():
    current_year = date.today().year
    silver_files = glob(f"../data/silver/{current_year}/raw/*/*/*.csv")
    
    unique_columns = set()

    if not silver_files:
        print("Nenhum arquivo encontrado na camada silver.")
        return set()

    for file in silver_files:
        try:
            df = pd.read_csv(file, nrows=5, encoding="utf-8-sig")
            unique_columns.update(df.columns.tolist())
        except Exception as e:
            print(f"Erro ao abrir {file}: {e}")

    return unique_columns


In [ ]:
cols = list_unique_silver_columns()

print("Colunas únicas encontradas:")
for col in sorted(cols):
    print("-", col)


- Renomera as colunas

In [ ]:
def standardize_silver_files():

    current_year = date.today().year
    silver_files = glob(f"../data/silver/{current_year}/raw/*/*/*.csv")
    

    rename_columns = {
    "DATA INICIAL": "data_inicial",
    "DATA FINAL": "data_final",
    "BRASIL": "pais",
    "ESTADO": "estado",
    "ESTADOS": "estado",
    "MUNICÍPIO": "municipio",
    "MUNICIPIO": "municipio",   # <- adicionado
    "REGIAO": "regiao",
    "PRODUTO": "produto",
    "NÚMERO DE POSTOS PESQUISADOS": "num_postos_pesquisados",
    "NUMERO DE POSTOS PESQUISADOS": "num_postos_pesquisados", # <- sem acento
    "UNIDADE DE MEDIDA": "unidade_medida",
    "PREÇO MÉDIO REVENDA": "preco_medio_revenda",
    "PRECO MEDIO REVENDA": "preco_medio_revenda",  # <- sem acento
    "DESVIO PADRÃO REVENDA": "desvio_padrao_revenda",
    "DESVIO PADRAO REVENDA": "desvio_padrao_revenda", # <- sem acento
    "PREÇO MÍNIMO REVENDA": "preco_minimo_revenda",
    "PRECO MINIMO REVENDA": "preco_minimo_revenda",
    "PREÇO MÁXIMO REVENDA": "preco_maximo_revenda",
    "PRECO MAXIMO REVENDA": "preco_maximo_revenda",
    "COEF DE VARIAÇÃO REVENDA": "coef_variacao_revenda",
    "COEF DE VARIACAO REVENDA": "coef_variacao_revenda"
}

    dfs = []  

    for file in silver_files:
        try:
            df = pd.read_csv(file, encoding="utf-8-sig")

            
            df.columns = [
                unicodedata.normalize("NFKD", col)
                .encode("ASCII", "ignore")
                .decode("utf-8")
                .strip()
                for col in df.columns
            ]

            
            df = df.rename(columns=rename_columns)

            
            df.columns = (
                df.columns
                .str.lower()
                .str.strip()
                .str.replace(" ", "_")
            )

            
            relative_path = file.replace(
                f"../data/silver/{current_year}/raw",
                f"../data/silver/{current_year}/raw_normalized/"
            )
            os.makedirs(os.path.dirname(relative_path), exist_ok=True)

            
            df.to_csv(relative_path, index=False, encoding="utf-8-sig")
            print(f"Arquivo padronizado salvo em: {relative_path}")

            
            dfs.append(df)

        except Exception as e:
            print(f"Erro ao processar {file}: {e}")

    
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()


In [ ]:
standardize_silver_files()

- Modelo Dimensional (Esquema Estrela)

  Dimensões:
  dim_produto

  dim_unidade

  dim_regiao

  dim_estado

  dim_municipio

  dim_capitais

  dim_pais

  dim_tempo
  
  dim_mes


Fatos:

fato_precos_capital

fato_precos_municipio

fato_precos_estado

fato_precos_regiao

In [ ]:
def load_silver_data():
    current_year = date.today().year
    silver_files = glob(f"../data/silver/{current_year}/raw_normalized/*/*/*.csv")

    dfs = []
    for file in silver_files:
        try:
            df_temp = pd.read_csv(
                file,
                encoding="utf-8-sig",
                na_values=["NaN", "nan", "NAN", ""]
            )
            dfs.append(df_temp)
        except Exception as e:
            print(f"Erro ao carregar {file}: {e}")

    if dfs:
        df_silver = pd.concat(dfs, ignore_index=True)
        return df_silver
    else:
        print("Nenhum arquivo encontrado na camada Silver.")
        return pd.DataFrame()



df_silver = load_silver_data()

print("Shape final:", df_silver.shape)
print("Colunas:", df_silver.columns.tolist())


In [ ]:
def create_dim_produto(df):
    produtos = (
        df["produto"]
        .dropna()
        .drop_duplicates()
        .reset_index(drop=True)
    )

    dim_produto = pd.DataFrame({
        "produto_id": range(1, len(produtos) + 1),
        "produto_descricao": produtos
    })

    return dim_produto



dim_produto = create_dim_produto(df_silver)


print(dim_produto.head(10))
print(f"Total de produtos distintos: {len(dim_produto)}")

In [ ]:
def create_dim_unidade(df):

    unidades = (
        df["unidade_medida"]
        .dropna()
        .drop_duplicates()
        .reset_index(drop=True)
    )

    dim_unidade = pd.DataFrame({
        "unidade_id": range(1, len(unidades) + 1),
        "unidade_descricao": unidades
    })

    return dim_unidade



dim_unidade = create_dim_unidade(df_silver)


print(dim_unidade.head())
print(f"Total de unidades distintas: {len(dim_unidade)}")


In [ ]:
def create_dim_tempo(df):
    datas = df[["data_inicial", "data_final"]].drop_duplicates().reset_index(drop=True)

    datas["tempo_id"] = range(1, len(datas) + 1)
    datas["ano"] = pd.to_datetime(datas["data_inicial"]).dt.year
    datas["mes_id"] = pd.to_datetime(datas["data_inicial"]).dt.month
    datas["semana"] = pd.to_datetime(datas["data_inicial"]).dt.isocalendar().week
    datas["dia"] = pd.to_datetime(datas["data_inicial"]).dt.day

    return datas[["tempo_id","data_inicial","data_final","ano","mes_id","semana","dia"]]


In [ ]:
def create_dim_mes():
    meses = pd.DataFrame({
        "mes_id": range(1, 13),
        "mes_descricao": pd.date_range("2025-01-01", periods=12, freq="MS").strftime("%B")
    })
    return meses


In [ ]:
def create_dim_pais(df):
    pais = df[["pais"]].dropna().drop_duplicates().reset_index(drop=True)
    pais["pais_id"] = range(1, len(pais) + 1)
    pais = pais.rename(columns={"pais":"pais_descricao"})
    return pais[["pais_id","pais_descricao"]]


In [ ]:
dim_tempo = create_dim_tempo(df_silver)
dim_mes = create_dim_mes()
dim_pais = create_dim_pais(df_silver)

print(dim_tempo.head())
print(dim_mes)
print(dim_pais)


In [ ]:
def create_dim_regiao(df):

    regiao = (
        df["regiao"]
        .replace(["NaN", "nan", "NAN", ""], pd.NA)  
        .dropna()
        .drop_duplicates()
        .reset_index(drop=True)
    )

    dim_regiao = pd.DataFrame({
        "regiao_id": range(1, len(regiao) + 1),
        "regiao_descricao": regiao
    })

    return dim_regiao



dim_regiao = create_dim_regiao(df_silver)


print(dim_regiao)


In [ ]:
def create_dim_estado(df, dim_regiao):

    estado = (
        df[["estado", "regiao"]]
        .dropna()
        .drop_duplicates()
        .reset_index(drop=True)
    )

    
    estado = estado.merge(
        dim_regiao,
        left_on="regiao",
        right_on="regiao_descricao",
        how="left"
    )

    
    estado["estado_id"] = range(1, len(estado) + 1)

    
    estado = estado.rename(columns={"estado": "estado_descricao"})

    return estado[["estado_id", "estado_descricao", "regiao_id"]]



dim_estado = create_dim_estado(df_silver, dim_regiao)


print(dim_estado.head())
print(f"Total de estados distintos: {len(dim_estado)}")


In [ ]:
def create_dim_municipio(df, dim_estado):

    capitais_list = [
        "RIO BRANCO","MACEIO","MACAPA","MANAUS","SALVADOR","FORTALEZA","BRASILIA",
        "VITORIA","GOIANIA","SAO LUIS","CUIABA","CAMPO GRANDE","BELO HORIZONTE",
        "BELEM","JOAO PESSOA","CURITIBA","RECIFE","TERESINA","RIO DE JANEIRO",
        "NATAL","PORTO ALEGRE","PORTO VELHO","BOA VISTA","FLORIANOPOLIS",
        "SAO PAULO","ARACAJU","PALMAS"
    ]

    
    municipio = (
        df.loc[df["municipio"].notna(), ["municipio","estado"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )

    
    municipio = municipio.merge(
        dim_estado[["estado_id","estado_descricao","regiao_id"]],
        left_on="estado", right_on="estado_descricao", how="left"
    )

    municipio["municipio_id"] = range(1, len(municipio)+1)
    municipio["is_capital"] = municipio["municipio"].isin(capitais_list).astype(int)

    
    dim_municipio = municipio.rename(columns={"municipio":"municipio_descricao"})
    dim_municipio = dim_municipio[["municipio_id","municipio_descricao","regiao_id","estado_id","is_capital"]]

    return dim_municipio


In [ ]:
dim_municipio = create_dim_municipio(df_silver, dim_estado)

print(dim_municipio.head(10))
print(f"Total municípios distintos: {len(dim_municipio)}")
print(f"Total capitais detectadas: {dim_municipio['is_capital'].sum()}")


In [ ]:
def save_dimensions(
    dim_produto, dim_unidade, dim_regiao, dim_estado,
    dim_municipio, dim_tempo, dim_mes, dim_pais
):
    current_year = date.today().year
    base_path = f"../data/gold/{current_year}/dim"
    os.makedirs(base_path, exist_ok=True)

    dim_produto.to_csv(f"{base_path}/dim_produto.csv", index=False, encoding="utf-8-sig")
    dim_unidade.to_csv(f"{base_path}/dim_unidade.csv", index=False, encoding="utf-8-sig")
    dim_regiao.to_csv(f"{base_path}/dim_regiao.csv", index=False, encoding="utf-8-sig")
    dim_estado.to_csv(f"{base_path}/dim_estado.csv", index=False, encoding="utf-8-sig")
    dim_municipio.to_csv(f"{base_path}/dim_municipio.csv", index=False, encoding="utf-8-sig")
    dim_tempo.to_csv(f"{base_path}/dim_tempo.csv", index=False, encoding="utf-8-sig")
    dim_mes.to_csv(f"{base_path}/dim_mes.csv", index=False, encoding="utf-8-sig")
    dim_pais.to_csv(f"{base_path}/dim_pais.csv", index=False, encoding="utf-8-sig")

    print(f"Todas as dimensões foram salvas em: {base_path}")


In [ ]:
save_dimensions(
    dim_produto, dim_unidade, dim_regiao, dim_estado,
    dim_municipio, dim_tempo, dim_mes, dim_pais
)

In [ ]:
current_year = date.today().year
dim_path = f"../data/gold/{current_year}/dim"

dim_produto    = pd.read_csv(f"{dim_path}/dim_produto.csv", encoding="utf-8-sig")
dim_unidade    = pd.read_csv(f"{dim_path}/dim_unidade.csv", encoding="utf-8-sig")
dim_regiao     = pd.read_csv(f"{dim_path}/dim_regiao.csv", encoding="utf-8-sig")
dim_estado     = pd.read_csv(f"{dim_path}/dim_estado.csv", encoding="utf-8-sig")
dim_municipio  = pd.read_csv(f"{dim_path}/dim_municipio.csv", encoding="utf-8-sig")
dim_tempo      = pd.read_csv(f"{dim_path}/dim_tempo.csv", encoding="utf-8-sig")
dim_mes        = pd.read_csv(f"{dim_path}/dim_mes.csv", encoding="utf-8-sig")
dim_pais       = pd.read_csv(f"{dim_path}/dim_pais.csv", encoding="utf-8-sig")

In [ ]:
silver_files = glob(f"../data/silver/{current_year}/raw_normalized/*/*/*.csv")
df_list = [pd.read_csv(file, encoding="utf-8-sig") for file in silver_files]
df_silver = pd.concat(df_list, ignore_index=True)


In [ ]:
for df, name, key in [
    (dim_produto, "dim_produto", "produto_id"),
    (dim_unidade, "dim_unidade", "unidade_id"),
    (dim_regiao, "dim_regiao", "regiao_id"),
    (dim_estado, "dim_estado", "estado_id"),
    (dim_municipio, "dim_municipio", "municipio_id"),
    (dim_tempo, "dim_tempo", "tempo_id"),
    (dim_mes, "dim_mes", "mes_id"),
    (dim_pais, "dim_pais", "pais_id"),
]:
    print(f"{name}: total={len(df)}, ids únicos={df[key].nunique()}, nulos={df[key].isna().sum()}")


In [ ]:
print("Estados sem região:", dim_estado[~dim_estado["regiao_id"].isin(dim_regiao["regiao_id"])])
print("Municípios sem estado:", dim_municipio[~dim_municipio["estado_id"].isin(dim_estado["estado_id"])])
print("Municípios sem região:", dim_municipio[~dim_municipio["regiao_id"].isin(dim_regiao["regiao_id"])])

In [ ]:
for col in ["produto_descricao","unidade_descricao","regiao_descricao",
            "estado_descricao","municipio_descricao","mes_descricao","pais_descricao"]:
    for df, name in [(dim_produto,"dim_produto"), (dim_unidade,"dim_unidade"),
                     (dim_regiao,"dim_regiao"), (dim_estado,"dim_estado"),
                     (dim_municipio,"dim_municipio"), (dim_mes,"dim_mes"), (dim_pais,"dim_pais")]:
        if col in df.columns:
            print(f"{name} - {col}: nulos = {df[col].isna().sum()}")


In [ ]:
print("dim_estado colunas:", dim_estado.columns.tolist())
print("dim_municipio colunas:", dim_municipio.columns.tolist())


In [414]:
def create_fato_precos_municipio(df, dim_produto, dim_unidade, dim_tempo,
                                 dim_pais, dim_estado, dim_municipio):
    
    fato = df.merge(
        dim_produto[["produto_id", "produto_descricao"]],
        left_on="produto", right_on="produto_descricao", how="left"
    )
    print("Após merge produto:", fato.columns.tolist())

    
    fato = fato.merge(
        dim_unidade[["unidade_id", "unidade_descricao"]],
        left_on="unidade_medida", right_on="unidade_descricao", how="left"
    )
    print("Após merge unidade:", fato.columns.tolist())

    
    fato = fato.merge(
        dim_tempo[["tempo_id","data_inicial","data_final","ano","mes_id","semana"]],
        on=["data_inicial","data_final"], how="left"
    )
    print("Após merge tempo:", fato.columns.tolist())

    
    fato["pais_id"] = 1

    
    fato = fato.merge(
        dim_estado[["estado_id","estado_norm","regiao_id"]],
        left_on="estado", right_on="estado_norm", how="left"
    )
    print("Após merge estado:", fato.columns.tolist())

    
    fato = fato.merge(
        dim_municipio[["municipio_id","municipio_norm","estado_id","regiao_id","is_capital"]],
        left_on="municipio", right_on="municipio_norm", how="left",
        suffixes=("_estado","_municipio")
    )
    print("Após merge municipio:", fato.columns.tolist())

    
    fato = fato.rename(columns={
        "estado_id_municipio": "estado_id",
        "regiao_id_municipio": "regiao_id"
    })

    fato = fato[[
        "data_inicial","data_final","ano","mes_id","semana",
        "produto_id","unidade_id","pais_id","regiao_id","estado_id","municipio_id",
        "is_capital","num_postos_pesquisados",
        "preco_medio_revenda","preco_minimo_revenda","preco_maximo_revenda",
        "desvio_padrao_revenda","coef_variacao_revenda"
    ]]

    
    fato = fato.dropna(subset=["estado_id","municipio_id","regiao_id"]).reset_index(drop=True)
    fato = fato.astype({
        "estado_id": "int64",
        "municipio_id": "int64",
        "regiao_id": "int64",
        "is_capital": "int64"
    })

    return fato


In [415]:
fato_municipio = create_fato_precos_municipio(
    df_silver,
    dim_produto,
    dim_unidade,
    dim_tempo,
    dim_pais,
    dim_estado,
    dim_municipio
)

print("\nFato Municipio pronto")
print(fato_municipio.head())
print("Tipos de dados:\n", fato_municipio.dtypes)
print("Total de linhas:", len(fato_municipio))


Após merge produto: ['data_inicial', 'data_final', 'pais', 'produto', 'num_postos_pesquisados', 'unidade_medida', 'preco_medio_revenda', 'desvio_padrao_revenda', 'preco_minimo_revenda', 'preco_maximo_revenda', 'coef_variacao_revenda', 'estado', 'municipio', 'regiao', 'produto_id', 'produto_descricao']
Após merge unidade: ['data_inicial', 'data_final', 'pais', 'produto', 'num_postos_pesquisados', 'unidade_medida', 'preco_medio_revenda', 'desvio_padrao_revenda', 'preco_minimo_revenda', 'preco_maximo_revenda', 'coef_variacao_revenda', 'estado', 'municipio', 'regiao', 'produto_id', 'produto_descricao', 'unidade_id', 'unidade_descricao']
Após merge tempo: ['data_inicial', 'data_final', 'pais', 'produto', 'num_postos_pesquisados', 'unidade_medida', 'preco_medio_revenda', 'desvio_padrao_revenda', 'preco_minimo_revenda', 'preco_maximo_revenda', 'coef_variacao_revenda', 'estado', 'municipio', 'regiao', 'produto_id', 'produto_descricao', 'unidade_id', 'unidade_descricao', 'tempo_id', 'ano', 'mes

In [416]:
def save_fato(df: pd.DataFrame, name: str):
    current_year = date.today().year
    base_path = f"../data/gold/{current_year}/fato"
    os.makedirs(base_path, exist_ok=True)

    file_path = os.path.join(base_path, f"{name}.csv")
    df.to_csv(file_path, index=False, encoding="utf-8-sig")

    print(f"Fato {name} salvo em {file_path}")

In [417]:
save_fato(fato_municipio, "fato_precos_municipio")


Fato fato_precos_municipio salvo em ../data/gold/2025/fato/fato_precos_municipio.csv


In [433]:
def create_fato_precos_estado(df, dim_produto, dim_unidade, dim_tempo, dim_estado, dim_regiao):
    fato = (
        df.merge(dim_produto[["produto_id", "produto_descricao"]],
                 left_on="produto", right_on="produto_descricao", how="left")
          .merge(dim_unidade[["unidade_id", "unidade_descricao"]],
                 left_on="unidade_medida", right_on="unidade_descricao", how="left")
          .merge(dim_tempo[["tempo_id","data_inicial","data_final","ano","mes_id","semana"]],
                 on=["data_inicial","data_final"], how="left")
          .merge(dim_estado[["estado_id","estado_norm","regiao_id"]],
                 left_on="estado", right_on="estado_norm", how="left")
    )

    
    fato["pais_id"] = 1

    
    fato_agg = fato.groupby(
        ["data_inicial","data_final","ano","mes_id","semana",
        "produto_id","unidade_id","pais_id","regiao_id","estado_id"],
        as_index=False
    ).agg({
        "num_postos_pesquisados": "sum",
        "preco_medio_revenda": "mean",
        "preco_minimo_revenda": "min",
        "preco_maximo_revenda": "max",
        "desvio_padrao_revenda": "mean",
        "coef_variacao_revenda": "mean"
    })

    fato_agg = fato_agg.astype({
        "estado_id": "int64",
        "regiao_id": "int64",
        "pais_id": "int64"
    })

    return fato_agg


In [434]:
fato_estado = create_fato_precos_estado(
    df_silver,
    dim_produto,
    dim_unidade,
    dim_tempo,
    dim_estado,
    dim_regiao
)

print(fato_estado.head())
print(fato_estado.dtypes)
print("Total de linhas:", len(fato_estado))


  data_inicial  data_final   ano  mes_id  semana  produto_id  unidade_id  \
0   2025-01-05  2025-01-11  2025       1       1           1           1   
1   2025-01-05  2025-01-11  2025       1       1           1           1   
2   2025-01-05  2025-01-11  2025       1       1           1           1   
3   2025-01-05  2025-01-11  2025       1       1           1           1   
4   2025-01-05  2025-01-11  2025       1       1           1           1   

   pais_id  regiao_id  estado_id  num_postos_pesquisados  preco_medio_revenda  \
0        1          1          1                       9              5.23250   
1        1          1          3                       6              5.11000   
2        1          1          4                     101              5.00250   
3        1          1         14                      75              4.85125   
4        1          1         22                      35              5.10200   

   preco_minimo_revenda  preco_maximo_revenda  desvio_pa

In [435]:
save_fato(fato_estado, "fato_precos_estado")


Fato fato_precos_estado salvo em ../data/gold/2025/fato/fato_precos_estado.csv


In [445]:
def create_fato_precos_regiao(df, dim_produto, dim_unidade, dim_tempo, dim_regiao):
    fato = (
        df.merge(dim_produto[["produto_id", "produto_descricao"]],
                 left_on="produto", right_on="produto_descricao", how="left")
          .merge(dim_unidade[["unidade_id", "unidade_descricao"]],
                 left_on="unidade_medida", right_on="unidade_descricao", how="left")
          .merge(dim_tempo[["tempo_id","data_inicial","data_final","ano","mes_id","semana"]],
                 on=["data_inicial","data_final"], how="left")
          .merge(dim_regiao[["regiao_id","regiao_descricao"]],
                 left_on="regiao", right_on="regiao_descricao", how="left")
    )

    
    fato["pais_id"] = 1

    
    fato_agg = fato.groupby(
        ["data_inicial","data_final","ano","mes_id","semana",
         "produto_id","unidade_id","pais_id","regiao_id"],
        as_index=False
    ).agg({
        "num_postos_pesquisados": "sum",
        "preco_medio_revenda": "mean",
        "preco_minimo_revenda": "min",
        "preco_maximo_revenda": "max",
        "desvio_padrao_revenda": "mean",      
        "coef_variacao_revenda": "mean"       
    })

    
    fato_agg = fato_agg.astype({
        "regiao_id": "int64",
        "pais_id": "int64"
    })

    return fato_agg


In [446]:
fato_regiao = create_fato_precos_regiao(
    df_silver,
    dim_produto,
    dim_unidade,
    dim_tempo,
    dim_regiao
)

print(fato_regiao.head())
print("Total de linhas:", len(fato_regiao))
print("Qtd regiões únicas:", fato_regiao["regiao_id"].nunique())


  data_inicial  data_final   ano  mes_id  semana  produto_id  unidade_id  \
0   2025-01-05  2025-01-11  2025       1       1           1           1   
1   2025-01-05  2025-01-11  2025       1       1           1           1   
2   2025-01-05  2025-01-11  2025       1       1           1           1   
3   2025-01-05  2025-01-11  2025       1       1           1           1   
4   2025-01-05  2025-01-11  2025       1       1           1           1   

   pais_id  regiao_id  num_postos_pesquisados  preco_medio_revenda  \
0        1          1                     260                4.980   
1        1          2                    1372                4.457   
2        1          3                     632                4.072   
3        1          4                    3792                4.214   
4        1          5                     928                4.475   

   preco_minimo_revenda  preco_maximo_revenda  desvio_padrao_revenda  \
0                  4.27                  5.74     

In [447]:
save_fato(fato_regiao, "fato_precos_regiao")


Fato fato_precos_regiao salvo em ../data/gold/2025/fato/fato_precos_regiao.csv


In [442]:
def create_fato_precos_pais(df, dim_produto, dim_unidade, dim_tempo, dim_pais=None):
    fato = (
        df.merge(dim_produto[["produto_id", "produto_descricao"]],
                 left_on="produto", right_on="produto_descricao", how="left")
          .merge(dim_unidade[["unidade_id", "unidade_descricao"]],
                 left_on="unidade_medida", right_on="unidade_descricao", how="left")
          .merge(dim_tempo[["tempo_id","data_inicial","data_final","ano","mes_id","semana"]],
                 on=["data_inicial","data_final"], how="left")
    )

    
    fato["pais_id"] = 1

    fato_agg = fato.groupby(
        ["data_inicial","data_final","ano","mes_id","semana",
         "produto_id","unidade_id","pais_id"],
        as_index=False
    ).agg({
        "num_postos_pesquisados": "sum",
        "preco_medio_revenda": "mean",
        "preco_minimo_revenda": "min",
        "preco_maximo_revenda": "max",
        "desvio_padrao_revenda": "mean",
        "coef_variacao_revenda": "mean"
    })

    fato_agg = fato_agg.astype({"pais_id": "int64"})

    return fato_agg


In [443]:
fato_pais = create_fato_precos_pais(
    df_silver,
    dim_produto,
    dim_unidade,
    dim_tempo
)

print(fato_pais.head())
print("Total de linhas:", len(fato_pais))
print("Países únicos:", fato_pais["pais_id"].nunique())


  data_inicial  data_final   ano  mes_id  semana  produto_id  unidade_id  \
0   2025-01-05  2025-01-11  2025       1       1           1           1   
1   2025-01-05  2025-01-11  2025       1       1           2           1   
2   2025-01-05  2025-01-11  2025       1       1           3           1   
3   2025-01-05  2025-01-11  2025       1       1           4           2   
4   2025-01-05  2025-01-11  2025       1       1           5           3   

   pais_id  num_postos_pesquisados  preco_medio_revenda  preco_minimo_revenda  \
0        1                   14708             4.391940                  3.29   
1        1                   13484             6.352258                  5.44   
2        1                   17456             6.179109                  5.04   
3        1                    7535           108.397400                 80.00   
4        1                    1596             4.877933                  3.79   

   preco_maximo_revenda  desvio_padrao_revenda  coef_var

In [444]:
save_fato(fato_pais, "fato_precos_pais")


Fato fato_precos_pais salvo em ../data/gold/2025/fato/fato_precos_pais.csv
